In [228]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import scipy
import math
import pandas_ta as ta
from pathlib import Path
import yfinance as yf
from datetime import datetime, timedelta, date

In [229]:
nas_path = Path('./nasdaq_2019_to_present.csv')
nas_df = pd.read_csv(nas_path)
nas_df['date'] = pd.to_datetime(nas_df['date'])
nas_df = nas_df.set_index('date')
nas_df.columns = ['close', 'open', 'high', 'low']
nas_df

,close,open,high,low
date,,,,
2024-01-21,17462.75,17462.75,17462.75,17462.75
2024-01-20,17462.75,17462.75,17462.75,17462.75
2024-01-19,17438.50,17116.00,17471.25,17107.25
2024-01-18,17110.00,16854.50,17125.75,16834.25
2024-01-17,16869.75,16978.00,16982.00,16689.25
...,...,...,...,...
2020-07-07,10532.25,10600.00,10694.50,10505.75
2020-07-06,10598.50,10347.25,10614.50,10341.50
2020-07-03,10328.88,10348.00,10389.25,10310.20


In [230]:
end_date = datetime.now() #adds current date

# Subtract 1825 days (5 years) from the end_date to get the start_date.
start_date = end_date - timedelta(days = 1825)

# Convert the start_date to a string in the format 'YYYY-MM-DD'.
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

In [231]:
security_df = yf.Ticker('NQ=F')

security_df.info # Accessing the information of the NASDAQ Futures. 

hist = security_df.history(start = start_date_str, end =  end_date_str, interval = '1D') # start = '2023-11-23', end = '2024-01-21'
                                                                 #Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
nasdaq_df = pd.DataFrame(hist[['Volume', 'Close', 'High', 'Low']])
df_cumulative_delta = pd.DataFrame(hist[['Volume', 'Close', 'High', 'Low']])
nasdaq_df


,Volume,Close,High,Low
Date,,,,
2019-01-23 00:00:00-05:00,515044,6665.25,6724.25,6592.25
2019-01-24 00:00:00-05:00,449017,6675.00,6718.50,6653.00
2019-01-25 00:00:00-05:00,446822,6791.25,6810.50,6676.75
2019-01-28 00:00:00-05:00,490045,6701.75,6791.00,6648.25
2019-01-29 00:00:00-05:00,441622,6638.75,6721.00,6616.75
...,...,...,...,...
2024-01-15 00:00:00-05:00,593445,16970.00,17024.25,16936.25
2024-01-16 00:00:00-05:00,816021,16966.50,17033.75,16811.75
2024-01-17 00:00:00-05:00,722784,16869.75,16982.00,16689.25


In [232]:
def find_levels(
        price: np.array,       # Array of log closing prices.
        atr: float,            # Average True Range (ATR), a measure of market volatility.
        first_w: float = 0.1,  # Initial weight for the weighted kernel density estimation.
        atr_mult: float = 3.0,  # Multiplier for ATR to adjust bandwidth in kernel density estimation.
        prom_thresh: float = 0.1  # Threshold for prominence in peak finding, relative to the maximum of the probability density function.
):

    # Setup for weights in the kernel density estimation.
    last_w = 1.0  # Final weight.
    w_step = (last_w - first_w) / len(price)  # Step size for weights based on the number of price points.
    weights = first_w + np.arange(len(price)) * w_step  # Linearly increasing weights array.
    weights[weights < 0] = 0.0  # Ensuring no negative weights.

    # Gaussian Kernel Density Estimation on the price data with weights and adjusted bandwidth.
    # Kernel Density Estimation is a non-parametric way to estimate the probability density function (PDF) of a random variable. 
    # In simple terms, it's a technique used to visualize the distribution of data points. 
    # The Gaussian KDE uses the Gaussian (normal) distribution as the kernel. 
    # This means it approximates the distribution of your data points by summing up Gaussian distributions (bell curves) centered at each data point.
    kernel = scipy.stats.gaussian_kde(price, bw_method=atr*atr_mult, weights=weights)

    # Construct market profile by evaluating the kernel density across a range of prices.
    min_v = np.min(price)  # Minimum price in the range.
    max_v = np.max(price)  # Maximum price in the range.
    step = (max_v - min_v) / 200  # Step size for the price range.
    price_range = np.arange(min_v, max_v, step)  # Price range for evaluation.
    pdf = kernel(price_range)  # Market profile as a probability density function.

    # Find significant peaks in the market profile.
    pdf_max = np.max(pdf)  # Maximum value in the probability density function.
    prom_min = pdf_max * prom_thresh  # Minimum prominence for a peak to be considered significant.

    # Finding peaks in the market profile with the defined minimum prominence.
    peaks, props = scipy.signal.find_peaks(pdf, prominence=prom_min)

    # Converting the log price back to the actual price at each peak to identify levels.
    levels = [np.exp(price_range[peak]) for peak in peaks]

    return levels, peaks, props, price_range, pdf, weights  # Return identified levels and related data.


def support_resistance_levels(
        data,                 # DataFrame containing the financial data.
        lookback: int,        # The number of periods to look back for calculating ATR and levels.
        first_w: float = 0.03,  # Initial weight for kernel density estimation in find_levels.
        atr_mult: float = 3.5,  # Multiplier for ATR to adjust bandwidth in kernel density estimation.
        prom_thresh: float = 0.20  # Threshold for prominence in peak finding.
):
    # Check if necessary columns are present in the DataFrame.
    if 'high' not in data.columns or 'low' not in data.columns or 'close' not in data.columns:
        raise KeyError("Columns 'high', 'low', and 'close' must be present in the DataFrame.")

    # Calculate the logarithmic Average True Range (ATR) for the given lookback period.
    atr = ta.atr(np.log(data['high']), np.log(data['low']), np.log(data['close']), lookback)

    # Initialize a list to store the levels for each data point.
    all_levels = [None] * len(data)

    # Iterate over the data, starting from the lookback period to the end of the data.
    for i in range(lookback, len(data)):
        i_start = i - lookback  # Determine the starting index for the lookback period.
        
        # Extract logarithmic closing prices for the lookback period.
        vals = np.log(data.iloc[i_start + 1: i + 1]['close'].to_numpy())

        # Calculate the support and resistance levels using the find_levels function.
        levels, peaks, props, price_range, pdf, weights = find_levels(
            vals, atr.iloc[i], first_w, atr_mult, prom_thresh
        )

        # Store the calculated levels for the current index.
        all_levels[i] = levels

    return all_levels  

def sr_penetration_signal(data, levels: list):
    # Initialize a signal array with zeros, same length as the data.
    signal = np.zeros(len(data))

    # Variable to store the current signal state.
    curr_sig = 0.0

    # Convert the 'close' column of the data to a numpy array for efficiency.
    close_arr = data['close'].to_numpy()

    # Iterate through each data point, starting from the second one (index 1).
    for i in range(1, len(data)):
        # If there are no levels identified for the current data point, skip to the next.
        if levels[i] is None:
            continue
        # Store the previous and current closing prices.
        last_c = close_arr[i - 1]
        curr_c = close_arr[i]
        # Check for penetration of each level identified for the current data point.
        for level in levels[i]:
            # If the current close is above the level and the last close was at or below it,
            # it indicates a bullish signal (cross above the support/resistance level).
            if curr_c > level and last_c <= level:
                curr_sig = 1.0
            # If the current close is below the level and the last close was at or above it,
            # it indicates a bearish signal (cross below the support/resistance level).
            elif curr_c < level and last_c >= level:
                curr_sig = -1.0

        # Update the signal array for the current index with the current signal state.
        signal[i] = curr_sig
    # Return the array containing the signals.
    return signal




def get_trades_from_signal(data, signal: np.array):
    # Initialize lists to store long and short trades.
    long_trades = []
    short_trades = []

    # Convert the 'close' column of the data to a numpy array for efficient processing.
    close_arr = data['close'].to_numpy()

    # Variable to store the last signal processed.
    last_sig = 0.0

    # Variable to keep track of the current open trade.
    open_trade = None

    # Get the index of the DataFrame, which represents time.
    idx = data.index

    # Iterate through each data point.
    for i in range(len(data)):
        # Check for a long entry signal and that it's not a continuation of the previous signal.
        if signal[i] == 1.0 and last_sig != 1.0:  # Long entry
            # Close any open short trade.
            if open_trade is not None:
                open_trade[2] = idx[i]  # Exit time.
                open_trade[3] = close_arr[i]  # Exit price.
                short_trades.append(open_trade)

            # Open a new long trade.
            open_trade = [idx[i], close_arr[i], -1, np.nan]  # Initialize with entry time and price.

        # Check for a short entry signal and that it's not a continuation of the previous signal.
        if signal[i] == -1.0 and last_sig != -1.0:  # Short entry
            # Close any open long trade.
            if open_trade is not None:
                open_trade[2] = idx[i]  # Exit time.
                open_trade[3] = close_arr[i]  # Exit price.
                long_trades.append(open_trade)

            # Open a new short trade.
            open_trade = [idx[i], close_arr[i], -1, np.nan]  # Initialize with entry time and price.

        # Update the last signal variable.
        last_sig = signal[i]

    # Convert the lists of trades to DataFrames for easier manipulation and analysis.
    long_trades = pd.DataFrame(long_trades, columns=['exit_time', 'entry_price', 'entry_time', 'exit_price'])
    short_trades = pd.DataFrame(short_trades, columns=['exit_time', 'entry_price', 'entry_time', 'exit_price'])

    # Calculate the percentage gain/loss for each trade.
    long_trades['percent'] = (long_trades['exit_price'] - long_trades['entry_price']) / long_trades['entry_price']
    short_trades['percent'] = -1 * (short_trades['exit_price'] - short_trades['entry_price']) / short_trades['entry_price']

    # Set the entry time as the index for both DataFrames.
    long_trades = long_trades.set_index('entry_time')
    short_trades = short_trades.set_index('entry_time')

    return long_trades, short_trades

In [233]:
if __name__ == '__main__': # This condition checks if the script is being run as the main program and not imported as a module.

    # Assigning 'nas_df' to 'data'. 'nas_df' should be a DataFrame containing financial data.
    data = nas_df

    # Set the plotting style to 'dark_background' for better visualization.
    plt.style.use('dark_background')

    # Calculate support and resistance levels using the 'support_resistance_levels' function.
    # It uses a 365-day lookback period, with specific parameters for weighted kernel density estimation.
    levels = support_resistance_levels(data, 365, first_w=1.0, atr_mult=3.0)

    # Generate trading signals based on the penetration of support and resistance levels.
    data['sr_signal'] = sr_penetration_signal(data, levels)

    # Calculate the logarithmic return of the 'close' price, shifting the result by -1 to align with the next period.
    data['log_ret'] = np.log(data['close']).diff().shift(-1)

    # Calculate the return from the strategy by multiplying the signal by the logarithmic return.
    data['sr_return'] = data['sr_signal'] * data['log_ret']
    
    # Generate lists of long and short trades based on the trading signals.
    long_trades, short_trades = get_trades_from_signal(data, data['sr_signal'].to_numpy())    

In [234]:
#Calcualting HLC3 value
nasdaq_df['HLC3'] = (nasdaq_df['High'] + nasdaq_df['Low'] + nasdaq_df['Close']) / 3
# Calculating VWAP (using HLC3)
nasdaq_df['VWAP'] = (nasdaq_df['HLC3'] * nasdaq_df['Volume']).cumsum() / nasdaq_df['Volume'].cumsum()
#Calculating Standard Deviation
nasdaq_df['PriceDeviation'] = (nasdaq_df['HLC3'] - nasdaq_df['VWAP']).rolling(window = 78).std() ##Window 78 set based on 5 min trading data (78 5 min in 1 day)##
# Session based (How it was on Pinescript) appears to be based on daily
# Calculate bands [UpperBand_1, LowerBand_1, UpperBand_2, LowerBand_2, UpperBand_3, LowerBand_3]
for i in [1, 2, 3]:
    nasdaq_df[f'UpperBand_{i}'] = nasdaq_df['VWAP'] + (i * nasdaq_df['PriceDeviation'])
    nasdaq_df[f'LowerBand_{i}'] = nasdaq_df['VWAP'] - (i * nasdaq_df['PriceDeviation'])
nasdaq_df = nasdaq_df.dropna()
df_hlc3_vwap = nasdaq_df
df_hlc3_vwap

,Volume,Close,High,Low,HLC3,VWAP,PriceDeviation,UpperBand_1,LowerBand_1,UpperBand_2,LowerBand_2,UpperBand_3,LowerBand_3
Date,,,,,,,,,,,,,
2019-05-14 00:00:00-04:00,547769,7421.00,7459.50,7290.00,7390.166667,7325.882796,173.655504,7499.538300,7152.227292,7673.193804,6978.571788,7846.849309,6804.916284
2019-05-15 00:00:00-04:00,535915,7529.25,7538.50,7353.75,7473.833333,7328.144559,171.243996,7499.388555,7156.900563,7670.632551,6985.656567,7841.876547,6814.412571
2019-05-16 00:00:00-04:00,525588,7600.25,7641.00,7473.50,7571.583333,7331.740453,168.286170,7500.026623,7163.454283,7668.312793,6995.168113,7836.598963,6826.881943
2019-05-17 00:00:00-04:00,626777,7511.50,7620.00,7506.50,7546.000000,7335.449321,166.404533,7501.853854,7169.044788,7668.258387,7002.640256,7834.662920,6836.235723
2019-05-20 00:00:00-04:00,634294,7391.75,7559.25,7361.50,7437.500000,7337.206244,164.701080,7501.907324,7172.505164,7666.608403,7007.804085,7831.309483,6843.103005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-15 00:00:00-05:00,593445,16970.00,17024.25,16936.25,16976.833333,12187.970022,767.447409,12955.417431,11420.522612,13722.864840,10653.075203,14490.312250,9885.627794
2024-01-16 00:00:00-05:00,816021,16966.50,17033.75,16811.75,16937.333333,12193.576902,769.949111,12963.526012,11423.627791,13733.475123,10653.678680,14503.424234,9883.729569
2024-01-17 00:00:00-05:00,722784,16869.75,16982.00,16689.25,16847.000000,12198.437746,769.139256,12967.577002,11429.298490,13736.716259,10660.159233,14505.855515,9891.019977


In [235]:
# Calculate the total points gained or lost in each long trade: (exit price - entry price).
long_trades['total_points'] = long_trades['exit_price'] - long_trades['entry_price']
# Calculate the profit and loss (PnL). Here, '20' is the point value per contract for the futures.
# Therefore, each point movement in the price results in a $20 gain or loss per contract.
long_trades['pnl_1_contract'] = long_trades['total_points'] * 20
long_trades['position'] = 'long'
long_trades

,exit_time,entry_price,exit_price,percent,total_points,pnl_1_contract,position
entry_time,,,,,,,
2022-08-09,2022-09-21,11710.00,13031.50,0.112852,1321.50,26430.0,long
2022-08-02,2022-08-05,13228.75,12924.50,-0.022999,-304.25,-6085.0,long
2022-07-06,2022-07-08,12152.00,11880.25,-0.022363,-271.75,-5435.0,long
2022-06-22,2022-06-29,11691.00,11565.75,-0.010713,-125.25,-2505.0,long
2022-03-28,2022-06-10,11840.00,14985.25,0.265646,3145.25,62905.0,long
2021-11-03,2021-11-08,16327.75,16129.75,-0.012127,-198.00,-3960.0,long
2021-09-30,2021-10-01,14761.75,14682.50,-0.005369,-79.25,-1585.0,long
2021-08-31,2021-09-29,14739.75,15582.50,0.057175,842.75,16855.0,long
2021-08-28,2021-08-30,15597.50,15428.25,-0.010851,-169.25,-3385.0,long


In [236]:
#calculate net profit of all long positions
long_pnl = long_trades['pnl_1_contract'].sum()
long_pnl

60590.0

In [237]:
# Calculate the total points gained or lost in each long trade.
short_trades['total_points'] = short_trades['entry_price'] - short_trades['exit_price']
# Calculate the profit and loss (PnL). Here, '20' is the point value per contract for the futures.
# Therefore, each point movement in the price results in a $20 gain or loss per contract.
short_trades['pnl_1_contract'] = short_trades['total_points'] * 20
short_trades['position'] = 'short'
short_trades

,exit_time,entry_price,exit_price,percent,total_points,pnl_1_contract,position
entry_time,,,,,,,
2022-08-05,2022-08-09,13031.50,13228.75,-0.015136,-197.25,-3945.0,short
2022-07-08,2022-08-02,12924.50,12152.00,0.059770,772.50,15450.0,short
2022-06-29,2022-07-06,11880.25,11691.00,0.015930,189.25,3785.0,short
2022-06-10,2022-06-22,11565.75,11840.00,-0.023712,-274.25,-5485.0,short
2021-11-08,2022-03-28,14985.25,16327.75,-0.089588,-1342.50,-26850.0,short
2021-10-01,2021-11-03,16129.75,14761.75,0.084812,1368.00,27360.0,short
2021-09-29,2021-09-30,14682.50,14739.75,-0.003899,-57.25,-1145.0,short
2021-08-30,2021-08-31,15582.50,15597.50,-0.000963,-15.00,-300.0,short
2021-07-29,2021-08-28,15428.25,15037.75,0.025311,390.50,7810.0,short


In [238]:
#calculate net profit of all short positions
short_pnl = short_trades['pnl_1_contract'].sum()
short_pnl

72955.0

In [239]:
# Concat the longs and shorts together
total_trades = pd.concat([long_trades, short_trades])
# order them by the first trade
total_trades.sort_values(by='entry_time', inplace=True)

total_trades.reset_index(inplace=True)
total_trades.rename(columns={'entry_time': 'entry'}, inplace=True)
# add a column for cumulative returns 

total_trades['entry'] = pd.to_datetime(total_trades['entry'])
total_trades['date'] = total_trades['entry']
total_trades = total_trades.set_index('date')

total_trades

,entry,exit_time,entry_price,exit_price,percent,total_points,pnl_1_contract,position
date,,,,,,,,
2020-08-05,2020-08-05,2020-08-06,11261.25,11091.75,-0.015052,-169.50,-3390.0,long
2020-08-06,2020-08-06,2020-08-25,11726.25,11261.25,0.039655,465.00,9300.0,short
2020-08-25,2020-08-25,2020-08-26,11991.25,11726.25,-0.022099,-265.00,-5300.0,long
2020-08-26,2020-08-26,2021-04-12,13808.75,11991.25,0.131619,1817.50,36350.0,short
2021-04-12,2021-04-12,2021-04-13,13975.75,13808.75,-0.011949,-167.00,-3340.0,long
2021-04-13,2021-04-13,2021-04-14,13798.75,13975.75,-0.012827,-177.00,-3540.0,short
2021-04-14,2021-04-14,2021-04-16,14029.50,13798.75,-0.016447,-230.75,-4615.0,long
2021-04-16,2021-04-16,2021-04-23,13927.00,14029.50,-0.007360,-102.50,-2050.0,short
2021-04-23,2021-04-23,2021-04-26,14011.50,13927.00,-0.006031,-84.50,-1690.0,long


In [240]:
nas_and_trades = pd.concat([total_trades, nas_df], axis=1)
nas_and_trades = nas_and_trades.drop(nas_and_trades.index[:23])
n = 23
nas_and_trades.drop(nas_and_trades.tail(n).index, inplace=True)
columns_to_drop = ['sr_signal', 'log_ret', 'sr_return']
nas_and_trades = nas_and_trades.drop(columns=columns_to_drop)
nas_and_trades['cumulative_pnl'] = nas_and_trades['pnl_1_contract'].fillna(0).cumsum() + 100000
nas_and_trades

,entry,exit_time,entry_price,exit_price,percent,total_points,pnl_1_contract,position,close,open,high,low,cumulative_pnl
date,,,,,,,,,,,,,
2020-08-03,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,11044.25,10933.25,11073.00,10876.25,100000.0
2020-08-04,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,11086.00,11054.00,11094.75,10983.00,100000.0
2020-08-05,2020-08-05,2020-08-06,11261.25,11091.75,-0.015052,-169.5,-3390.0,long,11091.75,11080.50,11144.00,11055.75,96610.0
2020-08-06,2020-08-06,2020-08-25,11726.25,11261.25,0.039655,465.0,9300.0,short,11261.25,11094.00,11271.75,11051.25,105910.0
2020-08-07,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,11122.75,11260.50,11283.25,11035.25,105910.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-24,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,16962.50,16962.50,16962.50,16962.00,233545.0
2023-12-25,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,17004.00,16991.75,17011.00,16985.00,233545.0
2023-12-26,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,17083.50,16964.50,17111.25,16964.50,233545.0


In [241]:
news_file = Path('C:\\Users\\Elliot\\Documents\\coding\\Berkeley\\project2\\calendar-event-list.csv')
news_df = pd.read_csv(news_file)
news_df.columns = ['id', 'date', 'name', 'impact', 'currency']
news_df['date'] = pd.to_datetime(news_df['date'])

news_df = news_df.set_index('date')
news_df.drop(news_df[news_df['impact'] == 'LOW'].index, inplace=True)
news_df.drop(news_df[news_df['impact'] == 'NONE'].index, inplace=True)
news_df = news_df.drop('id', axis=1)
news_df

,name,impact,currency
date,,,
2020-08-03 01:45:00,Caixin Manufacturing PMI,MEDIUM,CNY
2020-08-03 07:55:00,HCOB Manufacturing PMI,MEDIUM,EUR
2020-08-03 08:30:00,S&P Global/CIPS Manufacturing PMI,MEDIUM,GBP
2020-08-03 13:45:00,S&P Global Manufacturing PMI,MEDIUM,USD
2020-08-03 23:30:00,"Tokyo CPI ex Food, Energy (YoY)",MEDIUM,JPY
...,...,...,...
2024-01-24 14:45:00,BoC Interest Rate Decision,HIGH,CAD
2024-01-24 14:45:00,S&P Global Composite PMI,MEDIUM,USD
2024-01-24 14:45:00,S&P Global Manufacturing PMI,HIGH,USD


In [242]:
news_df.index = pd.to_datetime(news_df.index.date)

# Group by the modified 'date' index and aggregate the other columns
grouped_news = news_df.groupby(news_df.index).agg({
    'name': lambda x: list(x),
    'impact': lambda x: list(x),
    'currency': lambda x: list(x)
})

grouped_news = grouped_news.rename_axis('date')

# Display the resulting DataFrame
grouped_news

,name,impact,currency
date,,,
2020-08-03,"[Caixin Manufacturing PMI, HCOB Manufacturing ...","[MEDIUM, MEDIUM, MEDIUM, MEDIUM, MEDIUM, MEDIU...","[CNY, EUR, GBP, USD, JPY, JPY, JPY]"
2020-08-04,"[BoE MPC Vote Rate Unchanged, BoE MPC Vote Rat...","[HIGH, HIGH, MEDIUM, MEDIUM]","[GBP, GBP, CAD, NZD]"
2020-08-05,"[Caixin Services PMI, BoJ's Governor Kuroda sp...","[MEDIUM, HIGH, HIGH, MEDIUM, MEDIUM, MEDIUM, M...","[CNY, JPY, USD, USD, USD, USD, USD, USD, USD]"
2020-08-06,"[RBNZ Inflation Expectations (QoQ), BoE's Gove...","[MEDIUM, HIGH, HIGH, MEDIUM, MEDIUM, MEDIUM]","[NZD, GBP, GBP, USD, USD, USD]"
2020-08-10,"[Consumer Price Index (MoM), Consumer Price In...","[MEDIUM, HIGH, MEDIUM, MEDIUM]","[CNY, CNY, CNY, GBP]"
...,...,...,...
2024-01-18,"[Fed's Bostic speech, Fed's Bostic speech, Bus...","[MEDIUM, MEDIUM, MEDIUM, MEDIUM, MEDIUM, MEDIUM]","[USD, USD, NZD, JPY, JPY, JPY]"
2024-01-19,"[Producer Price Index (MoM), Producer Price In...","[MEDIUM, MEDIUM, MEDIUM, HIGH, MEDIUM, MEDIUM,...","[EUR, EUR, GBP, GBP, GBP, GBP, USD]"
2024-01-22,"[PBoC Interest Rate Decision, Business NZ PSI]","[MEDIUM, MEDIUM]","[CNY, NZD]"


In [243]:
news_nas_and_trades = pd.concat([nas_and_trades, grouped_news], axis=1)
c = 19
news_nas_and_trades.drop(news_nas_and_trades.tail(c).index, inplace=True)
news_nas_and_trades

,entry,exit_time,entry_price,exit_price,percent,total_points,pnl_1_contract,position,close,open,high,low,cumulative_pnl,name,impact,currency
date,,,,,,,,,,,,,,,,
2020-08-03,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,11044.25,10933.25,11073.00,10876.25,100000.0,"[Caixin Manufacturing PMI, HCOB Manufacturing ...","[MEDIUM, MEDIUM, MEDIUM, MEDIUM, MEDIUM, MEDIU...","[CNY, EUR, GBP, USD, JPY, JPY, JPY]"
2020-08-04,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,11086.00,11054.00,11094.75,10983.00,100000.0,"[BoE MPC Vote Rate Unchanged, BoE MPC Vote Rat...","[HIGH, HIGH, MEDIUM, MEDIUM]","[GBP, GBP, CAD, NZD]"
2020-08-05,2020-08-05,2020-08-06,11261.25,11091.75,-0.015052,-169.5,-3390.0,long,11091.75,11080.50,11144.00,11055.75,96610.0,"[Caixin Services PMI, BoJ's Governor Kuroda sp...","[MEDIUM, HIGH, HIGH, MEDIUM, MEDIUM, MEDIUM, M...","[CNY, JPY, USD, USD, USD, USD, USD, USD, USD]"
2020-08-06,2020-08-06,2020-08-25,11726.25,11261.25,0.039655,465.0,9300.0,short,11261.25,11094.00,11271.75,11051.25,105910.0,"[RBNZ Inflation Expectations (QoQ), BoE's Gove...","[MEDIUM, HIGH, HIGH, MEDIUM, MEDIUM, MEDIUM]","[NZD, GBP, GBP, USD, USD, USD]"
2020-08-07,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,11122.75,11260.50,11283.25,11035.25,105910.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-24,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,16962.50,16962.50,16962.50,16962.00,233545.0,NaN,NaN,NaN
2023-12-25,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,17004.00,16991.75,17011.00,16985.00,233545.0,NaN,NaN,NaN
2023-12-26,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,17083.50,16964.50,17111.25,16964.50,233545.0,NaN,NaN,NaN
